# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*csv'))
    #print(file_path_list)

/home/workspace


In [3]:
file_path_list[:2]

['/home/workspace/event_data/2018-11-30-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv']

#### Using pandas for processing files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
col_list = ['artist',
           'firstName',
           'gender',
           'itemInSession',
           'lastName',
           'length',
           'level',
           'location',
           'sessionId',
           'song',
           'userId']

#creat an empty csv with the required columns
pd.DataFrame(columns= col_list).to_csv('event_datafile_new_1.csv', index = False)

#loop through every file and append data in the csv file
for file_path in file_path_list:
    #read the file
    df = pd.read_csv(file_path, usecols= col_list)
    
    #append the data 
    df[~df.artist.isna()].to_csv('event_datafile_new_1.csv', mode='a', header = False, index = False)

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new_1.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
#Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)


#### Set Keyspace

In [8]:
#Setting KEYSPACE to the keyspace specified above
##def pandas_factory(colnames, rows):
    ##return pd.DataFrame(rows, columns=colnames)

##try:
session.set_keyspace('udacity')
    ##session.row_factory = pandas_factory
##except Exception as e:
   ## print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4





   
    SELECT
       artist_name, 
       song_title,
       song_length
       
    FROM
       song_playlist_item_session
    WHERE 
       session_id = 338
    AND
       item_in_session = 4  



In [9]:
#Creating Query 1 Table

query = "CREATE TABLE IF NOT EXISTS song_playlist_item_session"
query += '''(
              session_id int,
              item_in_session int, 
              artist_name text, 
              song_length float, 
              song_title text, 
              
              PRIMARY KEY(session_id, item_in_session)
              )'''
# PRIMARY KEY: combining session_id and item_in_session partion key will make the row unique (itemInSession are clustering keys.)
# we used two partion keys in the where class in the mentioned above order.(rows are ordered by the itemInSession.)

try:
    session.execute(query)
except Exception as e:
    print(e)


In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new_1.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_item_session (session_id, item_in_session, artist_name,  song_title , song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))  
        #print(line)
        #break

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
#Add in the SELECT statement to verify the data was entered into the table

query = """
SELECT 
   artist_name,
   song_title, 
   song_length    
   
FROM
   song_playlist_item_session
WHERE 
   session_id = 338
AND
   item_in_session = 4 
"""
rows = session.execute(query)
for row in rows:
    print(row.artist_name, row. song_title, row.song_length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875





### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    






   
    SELECT
     artist_name,
     song_title,
     first_name,
     last_name
       
    FROM
       user_activity
    WHERE 
       user_id = 10
    AND
       session_id = 182
    ORDER BY 
       item_in_session DESC


In [12]:
#Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "DROP TABLE IF EXISTS user_activity"
try:
    session.execute(query)
except Exception as e:
    print(e)

## Here the Primary Key has three fields: userid and sessionid are the partition key, and itemInSession are clustering keys. 
## Partitioning is done by userid and sessionid, and within that partition, rows are ordered by the itemInSession.
query = "CREATE TABLE IF NOT EXISTS user_activity "
query = query + "(user_id float, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text," 
query = query + " PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
#Assign the INSERT statements into the `query` variable
file = 'event_datafile_new_1.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_activity (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (float(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """   
SELECT
     artist_name,
     song_title,
     first_name,
     last_name
       
FROM
    user_activity
WHERE 
   user_id = 10
AND
   session_id = 182  
"""
rows = session.execute(query)
for row in rows:
    print(row.artist_name, row. song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz



### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

   
    SELECT
     user_id,
     first_name,
     last_name
       
    FROM
       song_playlist_library
    WHERE 
       song_title = 'All Hands Against His Own'
    


In [15]:
#Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "DROP TABLE IF EXISTS song_playlist_library"
try:
    session.execute(query)
except Exception as e:
    print(e)

## Here the Primary Key has two fields: song is the partition key, and userid are clustering keys. 
## Partitioning is done by song and within that partition, rows are ordered by the userid.
query = "CREATE TABLE IF NOT EXISTS song_playlist_library "
query = query + "(song_title text, user_id float, first_name text, last_name text," 
query = query + " PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                

In [17]:
#Assign the INSERT statements into the `query` variable
file = 'event_datafile_new_1.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_library (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], float(line[10]), line[1], line[4]))
                        

#### Do a SELECT to verify that the data have been inserted into each table

In [20]:
#Add in the SELECT statement to verify the data was entered into the table

query = """
SELECT
     user_id,
     first_name,
     last_name
FROM
    song_playlist_library
WHERE 
    song_title = 'All Hands Against His Own'
       
"""
rows = session.execute(query)
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
#Drop the table before closing out the sessions

query = "DROP TABLE IF EXISTS song_playlist_item_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_activity"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_playlist_library"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()